# JS04 - Working with Graphs

## Overview

Today’s applications are required to be highly responsive and always online. They need to be deployed in data centers closer to their users and can access data instantly across the globe.

Macrometa global data network (GDN) is a fully managed real-time materialized view engine that provides access to data instantly to Apps & APIs in a single and straightforward interface.

This article introduces working with documents in GDN with jsC8 drivers.

In the drivers, a document is a dictionary/object that is JSON serializable with the following properties:

- It contains the `_key` field, which identifies the document uniquely within a specific collection.
- It contains the `_id` field (also called the handle), which identifies the document uniquely across all collections within a fabric. This ID is a combination of the collection name and the document key using the format `{collection}/{key}` (see example below).
- It contains the `_rev` field. GDN supports MVCC (Multiple Version Concurrency Control) and is capable of storing each document in multiple revisions. This field indicates the latest revision of a document. The field is populated by GDN and is not required as input unless you want to validate a document against its current revision.

Here is an example of a valid document:

In [ ]:
{
    '_id': 'students/bruce',
    '_key': 'bruce',
    '_rev': '_Wm3dzEi--_',
    'first_name': 'Bruce',
    'last_name': 'Wayne',
    'address': {
        'street' : '1007 Mountain Dr.',
        'city': 'Gotham',
        'state': 'NJ'
    },
    'is_rich': True,
    'friends': ['robin', 'gordon']
}

Edge documents (edges) are similar to standard documents but with two additional required fields `_from` and `_to`. Values of these fields must be the handles of "from" and "to" vertex documents linked by the edge document in question. Here is an example of a valid edge document:

In [ ]:
{
    '_id': 'friends/001',
    '_key': '001',
    '_rev': '_Wm3dyle--_',
    '_from': 'students/john',
    '_to': 'students/jane',
    'closeness': 9.5
}

A Graph consists of vertices and edges. Edges are stored as documents in edge collections. A vertex can be a document of a document collection or of an edge collection (so edges can be used as vertices). Which collections are used within a named graph is defined via edge definitions. A named graph can contain more than one edge definition; at least one is needed. Graphs allow you to structure your models in line with your domain and group them logically in collections, giving you the power to query them in the same graph queries.

In SQL, you commonly have the construct of a relation table to store n:m relations between two data tables. An edge collection is somewhat similar to these relation tables. Vertex collections resemble the data tables with the objects to connect.

While simple graph queries with a fixed number of hops via the relation table may be doable in SQL with several nested joins, graph databases can handle an arbitrary number of these hops over edge collections - this is called traversal. Also, edges in one edge collection may point to several vertex collections. Finally, it's common to have attributes attached to edges, i.e., a label naming this interconnection.

Edges have a direction, with their relations `_from` and `_to` pointing from one document to another document stored in vertex collections. In queries, you can define in which directions the edge relations may be followed, i.e.,

- OUTBOUND: `_from` → `_to`
- INBOUND: `_from` ← `_to`
- ANY: `_from` ↔ `_to`

## Pre-requisite

Lets Assume 
- You have already made a tenant account, and have a username and password
- You have installed the jsC8 drivers as explained in section 01
- You have generated an API Key as explained in section 01


In [ ]:
#/* run this once to install javascript kernal and jsc8 in google colab, then reload, and then skip this
!npm install jsc8
!npm install -g --unsafe-perm ijavascript
!ijsinstall --install=global  # */

## 1. Connect to GDN

In [ ]:
let emailId, password;
let fed_url = "https://gdn.paas.macrometa.io";

In [ ]:
$$.input(
  { prompt: "Please, type fedration url, leave blank to use default and press enter: " },
  (error, url) => {
      if(url)
        fed_url = url;
    $$.done();
  }
);

In [ ]:
$$.input(
  { prompt: "Please, type your email and press enter: " },
  (error, email) => {
    emailId = email;
    $$.done();
  }
);

In [ ]:
$$.input(
  { prompt: "Please, type your password and press enter: ", password: true },
  (error, pass) => {
    password = pass;
    $$.done();
  }
);

In [ ]:
const jsc8 = require("jsc8");

// ----- simple way  -----
const client = new jsc8({
  url: fed_url,
});

client
  .login(emailId, password)
  .then((result) => console.log("Login successfully", result))
  .catch((err) => console.error("Error while login", err.message));

// ----- with token -----
/* const client = new jsc8({
  url: "https://gdn.paas.macrometa.io",
  token: "XXXX",
  fabricName: "_system",
}); */

// ----- with apikey-----
/* const client = new jsc8({
  url: "https://gdn.paas.macrometa.io",
  apiKey: "XXXX",
  fabricName: "_system",
}); */

## 2. Get GeoFabric Details

To get the details of fabric:

In [ ]:
// you might not need this, but if you wanted to select a
// specific GeoFabric you can find out whats available by executing this code!

const getGeoFabricMetaData = async () => {
  console.log("Get geo fabric details...");

  try {
    const fabrics = await client.get();
    console.log(fabrics);
  } catch (err) {
    console.error("Error while fetching GeoFabric details", err.message);
  }
};

getGeoFabricMetaData();

## 3. Create Collection

We can now create a collection in the fabric. First, you connect to fabric and then create a collection called employees.

The below example shows the steps:

In [ ]:
const collection_name = "students";

const createCollection = async () => {
  let collectionDetails;
  try {
    const isCollectionExists = await client.hasCollection(collection_name);
    if (isCollectionExists) {
      console.log("Collection exists with name ", collection_name);
      return;
    }
    collectionDetails = await client.createKVCollection(collection_name);
    console.log("Collection created! ", collection_name);
  } catch (e) {
    return "Collection creation did not succeed due to " + e;
  }
};

createCollection();

## 4. Create an Edge Collection

An edge collection contains edge documents and shares its namespace with all other types of collections. You can manage edge documents via standard collection API wrappers, but using edge collection API wrappers provides additional safeguards:

- All modifications are executed in transactions
- Edge documents are checked against the edge definitions upon insert

In [ ]:
const edge_name = "school";

const createEdgeCollection = async () => {
  let collectionDetails;
  try {
    const isCollectionExists = await client.hasCollection(edge_name);
    if (isCollectionExists) {
      console.log("Graph exists with name ", edge_name);
      return;
    }
    collectionDetails = await client.createCollection(edge_name, {}, true);
    console.log("Created Edge Collection! ", edge_name);
  } catch (err) {
    console.error("Collection creation did not succeed due to " + err.message);
  }
};

createEdgeCollection();

## 5. Insert Documents

Let's insert documents to the students collection as shown below:

In [ ]:
let docs = [
  {
    _key: "Jenny",
    firstname: "Jenney",
    lastname: "Jones",
    email: "email",
  },
  {
    _key: "Bob",
    firstname: "Bob",
    lastname: "Billy",
    email: "email",
  },
  {
    _key: "Alan",
    firstname: "Alan",
    lastname: "Evans",
    email: "email",
  },
];

const insertDocument = async () => {
  try {
    await client.insertDocument(collection_name, docs);
    console.log("Documents inserted");
  } catch (err) {
    console.error("Error while inserting documents", err.message);
  }
};

insertDocument();

## 6. Create Graph

A graph consists of vertices and edges. Vertices are stored as documents in vertex collections, and edges are stored as documents in edge collections. The collections used in a graph and their relations are specified with edge definitions.

In [ ]:
const createGraph = async () => {
  try {
    const isGraphExists = await client.hasGraph("school");
    if (isGraphExists) {
      console.log("Graph exists with name school");
      return;
    }
    collectionDetails = await client.createGraph("school");
    console.log("Graph Created");
  } catch (err) {
    console.error("Graph creation did not succeed due to ", err.message);
  }
};

createGraph();

## 7. Time to tidy up!

That was great! Now let's tidy up by removing the graph we created.

In [ ]:
const deleteCollection = async () => {
  try {
    await client.deleteCollection(collection_name);
    console.log("Collection Deleted: ", collection_name);
    await client.deleteCollection(edge_name);
    console.log("Collection Deleted: ", edge_name);
  } catch (err) {
    console.error("Error while deleting collection ", err.message);
  }
};

deleteCollection();

In [ ]:
const deleteGraphData = async () => {
  try {
    console.log("Delete graph");
    await client.deleteGraph("school", true);
    console.log("Collection and Graph deleted");
  } catch (err) {
    console.error("Error while deleting collection and graphs:", err.message);
  }
};

deleteGraphData();

## Section Completed!

TBC...